In [ ]:
%pip install -r requirements.txt

In [2]:
# -------------------------------
# Imports
# -------------------------------

# Standard Library
import json
import os
import re
import threading
import time
from datetime import datetime

# Third-party packages
import dash
from dash import dcc, html
from dash.dependencies import Input as DashInput, Output
import ipywidgets as widgets
import numpy as np
import pandas as pd
import plotly.graph_objs as go
import requests
from IPython.display import display, clear_output, HTML
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from tensorflow.keras.layers import Conv1D, Dense, Dropout, Input, LSTM
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.optimizers import Adam


# -------------------------------
# Settings laden / default erstellen
# -------------------------------
settings_file = "settings.json"
if not os.path.exists(settings_file):
    default_settings = {
        "allgemein_settings": {
            "model_folder": "models",
            "use_model_file": "Heusc_v0.3_20250913_024701.keras",
            "sequence_length": 60,
            "train_epochs": 10000,
            "batch_size": 32,
            "validation_split": 0.2,
            "early_stopping_patience": 5,
            "feature_scaling": True,
            "use_dropout": True,
            "dropout_rate": 0.2,
            "loss_function": "binary_crossentropy",
            "optimizer": "adam",
            "debug_mode": True
        },
        "offline": {
            "enabled": True,
            "csv_folder": "csv/binance",
            "csv_file": "BTCUSDT-1m-1y-binance-2025-09-12_22-40-01.csv"
        },
        "online": {
            "symbols": ["BTC","ETH","SOL"],
            "currencies": ["USDT","USDT","USDT"],
            "interval": "1m",
            "poll_seconds": 10,
            "max_live_train_minutes": 60
        },
        "balance": {
            "use_simulation": True,
            "initial_balance": 1000,
            "save_on_profit_percent": 50,
            "game_over_threshold": 0,
            "balance_reward_factor": 1.0,
            "retrain_from_best": True
        },
        "training_options": {
            "use_weighting": True,
            "punish_on_wrong": True,
            "predict_confidence": True,
            "log_transactions": True,
            "continual_learning": True,
            "transfer_learning": True,
            "ensemble_enabled": True
        }
    }
    with open(settings_file,"w") as f:
        json.dump(default_settings, f, indent=4)

with open(settings_file,"r") as f:
    settings = json.load(f)

allg = settings['allgemein_settings']
online = settings['online']
offline = settings['offline']
balance_settings = settings['balance']
training_options = settings['training_options']

print(f"allg: {allg}")
print(f"online: {online}")
print(f"offline: {offline}")
print(f"balance_settings: {balance_settings}")
print(f"training_opts: {training_options}")



allg: {'model_folder': 'models', 'use_model_file': '', 'sequence_length': 128, 'train_epochs': 1000, 'batch_size': 64, 'validation_split': 0.2, 'early_stopping_patience': 5, 'feature_scaling': True, 'use_dropout': True, 'dropout_rate': 0.2, 'loss_function': 'binary_crossentropy', 'optimizer': 'adam', 'debug_mode': True}
online: {'symbols': ['BTC', 'ETH', 'SOL'], 'currencies': ['USDT', 'USDT', 'USDT'], 'interval': '1m', 'poll_seconds': 10, 'max_live_train_minutes': 60}
offline: {'enabled': True, 'csv_folder': 'csv/binance', 'csv_file': 'BTCUSDT-1m-1mo-binance-2025-09-12_22-43-22.csv'}
balance_settings: {'use_simulation': True, 'initial_balance': 1000, 'save_on_profit_percent': 50, 'game_over_threshold': 0, 'balance_reward_factor': 1.0, 'retrain_from_best': True}
training_opts: {'use_weighting': True, 'punish_on_wrong': True, 'predict_confidence': True, 'log_transactions': True, 'continual_learning': True, 'transfer_learning': True, 'ensemble_enabled': True}


In [3]:
# Startordner für CSVs (relativer Pfad)
notebook_dir = os.getcwd()
start_folder = os.path.abspath(os.path.join(notebook_dir, f"../../{settings['offline']['csv_folder']}"))
os.makedirs(start_folder, exist_ok=True)

# Widgets
folder_dropdown = widgets.Dropdown(description="Ordner:", layout=widgets.Layout(width="70%"))
file_dropdown = widgets.Dropdown(description="Datei:", layout=widgets.Layout(width="70%"))
save_button = widgets.Button(description="Speichern", button_style="success")
output = widgets.Output()

def update_folder_options(folder):
    items = [d for d in os.listdir(folder) if os.path.isdir(os.path.join(folder, d))]
    folder_dropdown.options = [".."] + items
    folder_dropdown.value = None
    
    files = [f for f in os.listdir(folder) if f.endswith(".csv")]
    if not files:
        files = ["<keine CSV gefunden>"]
    file_dropdown.options = files
    if settings['offline'].get('csv_file') in files:
        file_dropdown.value = settings['offline']['csv_file']
    else:
        file_dropdown.value = files[0]

def folder_changed(change):
    if change['new'] is not None:
        selected = change['new']
        new_folder = os.path.normpath(os.path.join(current_folder[0], ".." if selected==".." else selected))
        if os.path.exists(new_folder):
            current_folder[0] = new_folder
            update_folder_options(new_folder)
            with output:
                clear_output()
                print(f"[INFO] Aktueller Ordner: {new_folder}")
                print(f"[INFO] Aktuelle CSV: {file_dropdown.value}")

def save_csv(b):
    selected_file = file_dropdown.value
    if selected_file != "<keine CSV gefunden>":
        settings['offline']['csv_file'] = selected_file
        with open(settings_file, "w") as f:
            json.dump(settings, f, indent=4)
        with output:
            clear_output()
            print(f"[INFO] CSV-Datei in settings.json auf '{selected_file}' gesetzt")
    else:
        with output:
            clear_output()
            print("[WARN] Keine CSV-Datei zum Speichern vorhanden!")

current_folder = [start_folder]
update_folder_options(start_folder)

folder_dropdown.observe(folder_changed, names='value')
save_button.on_click(save_csv)

display(folder_dropdown, file_dropdown, save_button, output)


Dropdown(description='Ordner:', layout=Layout(width='70%'), options=('..',), value=None)

Dropdown(description='Datei:', index=3, layout=Layout(width='70%'), options=('BTCUSDT-1m-3mo-binance-2025-09-1…

Button(button_style='success', description='Speichern', style=ButtonStyle())

Output()

In [4]:
# Absoluten Pfad zur CSV-Datei erstellen
csv_folder = os.path.abspath(os.path.join(notebook_dir, f"../../{offline['csv_folder']}"))
csv_file = offline['csv_file']  # z.B. 'BTCUSDT-1m-1y-binance-2025-09-13_11-37-53.csv'

csv_file_path = os.path.join(csv_folder, csv_file)

print(f"Pfad zur CSV: {csv_file_path}")
print(f"Existiert Datei? {os.path.exists(csv_file_path)}")  # True = OK

def load_csv_data(file_path, sequence_length=60, feature_scaling=True):
    # CSV laden
    df = pd.read_csv(file_path)
    
    # Timestamp in datetime umwandeln
    if 'timestamp' not in df.columns:
        raise ValueError("CSV enthält keine 'timestamp'-Spalte!")
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df.sort_values('timestamp', inplace=True)

    # Wichtige Features auswählen
    required_features = ['open', 'high', 'low', 'close', 'volume']
    missing_features = [f for f in required_features if f not in df.columns]
    if missing_features:
        raise ValueError(f"CSV fehlt benötigte Spalten: {missing_features}")
    
    # Fehlende Werte entfernen
    df = df.dropna(subset=required_features)
    
    data = df[required_features].values
    
    # Feature Scaling optional
    scaler = None
    if feature_scaling:
        scaler = StandardScaler()
        data = scaler.fit_transform(data)

    # Sequenzen und Labels erstellen
    sequences, labels = [], []
    for i in range(len(data) - sequence_length):
        sequences.append(data[i:i+sequence_length])
        # Label: 1 wenn Close der nächsten Kerze steigt, sonst 0
        labels.append(1 if data[i+sequence_length][3] > data[i+sequence_length-1][3] else 0)

    sequences = np.array(sequences)
    labels = np.array(labels)

    return sequences, labels, scaler

# Beispiel Nutzung
if offline['enabled']:
    csv_folder = os.path.abspath(os.path.join(notebook_dir, f"../../{offline['csv_folder']}"))
    csv_file = offline['csv_file']
    csv_file_path = os.path.join(csv_folder, csv_file)

    if os.path.exists(csv_file_path):
        seq_len = allg['sequence_length']
        X, y, scaler = load_csv_data(csv_file_path, sequence_length=seq_len, feature_scaling=allg['feature_scaling'])
        print(f"Data shape: X={X.shape}, y={y.shape}")
    else:
        print("[ERROR] CSV-Datei nicht gefunden! Bitte Pfad überprüfen.")

Pfad zur CSV: d:\.Projekt\Programmieren\Python\AI\Project HEUSC\Konzept_One\Daytrading\v1.0\csv\binance\BTCUSDT-1m-1mo-binance-2025-09-12_22-43-22.csv
Existiert Datei? True
Data shape: X=(42908, 128, 5), y=(42908,)


In [5]:
def create_cnn_lstm_model(sequence_length, n_features, dropout_rate=0.2, use_dropout=True):
    model = Sequential()
    # Convolutional Layer
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(sequence_length, n_features)))
    model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
    
    # LSTM Layer
    model.add(LSTM(50, return_sequences=False))
    
    if use_dropout:
        model.add(Dropout(dropout_rate))
    
    # Dense Output
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [6]:
def create_big_cnn_lstm(sequence_length, n_features, dropout_rate=0.3, use_dropout=True):
    model = Sequential()
    
    # CNN Block (zweifach)
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(sequence_length, n_features)))
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
    
    # LSTM Block (dreifach)
    model.add(LSTM(128, return_sequences=True))
    model.add(LSTM(128, return_sequences=True))
    model.add(LSTM(128, return_sequences=False))
    
    # Optional Dropout
    if use_dropout:
        model.add(Dropout(dropout_rate))
    
    # Dense Output
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(
        optimizer=Adam(learning_rate=0.0001),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model


In [7]:
def   create_small_cnn_lstm(sequence_length, n_features, dropout_rate=0.2, use_dropout=True):
    model = Sequential()
    
    # CNN Block (kleiner)
    model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(sequence_length, n_features)))
    
    # LSTM Block (dreifach, aber kleiner)
    model.add(LSTM(64, return_sequences=True))
    model.add(LSTM(64, return_sequences=True))
    model.add(LSTM(64, return_sequences=False))
    
    # Optional Dropout
    if use_dropout:
        model.add(Dropout(dropout_rate))
    
    # Dense Output
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(
        optimizer=Adam(learning_rate=5e-4),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model


In [8]:
def train_ensemble_models(X, y, seq_len, n_features, allg):
    # --- Modelle erstellen ---
    big_model = create_big_cnn_lstm(seq_len, n_features, 
                                    dropout_rate=allg['dropout_rate'], 
                                    use_dropout=allg['use_dropout'])
    
    small_model = create_small_cnn_lstm(seq_len, n_features, 
                                        dropout_rate=allg['dropout_rate'], 
                                        use_dropout=allg['use_dropout'])
    
    # --- EarlyStopping ---
    early_stop = EarlyStopping(
        monitor='val_loss',
        patience=allg['early_stopping_patience'],
        restore_best_weights=True
    )
    
    # --- Trainieren ---
    print("🔹 Training Big Model...")
    history_big = big_model.fit(
        X, y,
        epochs=allg['train_epochs'],
        batch_size=allg['batch_size'],
        validation_split=allg['validation_split'],
        callbacks=[early_stop],
        verbose=1
    )
    
    print("🔹 Training Small Model...")
    history_small = small_model.fit(
        X, y,
        epochs=allg['train_epochs'],
        batch_size=allg['batch_size'],
        validation_split=allg['validation_split'],
        callbacks=[early_stop],
        verbose=1
    )
    
    return big_model, small_model, history_big, history_small


In [9]:

def ensemble_predict(big_model, small_model, X, method="average"):
    pred_big = big_model.predict(X, verbose=0)
    pred_small = small_model.predict(X, verbose=0)
    
    if method == "average":
        return (pred_big + pred_small) / 2
    elif method == "weighted":
        # Gewichtung: Big Model wichtiger
        return (0.7 * pred_big + 0.3 * pred_small)
    else:
        raise ValueError("Unknown method, use 'average' or 'weighted'")


In [13]:
#First V1
# Modell erstellen
n_features = X.shape[2]
model = create_cnn_lstm_model(seq_len, n_features, dropout_rate=allg['dropout_rate'], use_dropout=allg['use_dropout'])
model.summary()

# EarlyStopping
early_stop = EarlyStopping(monitor='val_loss', patience=allg['early_stopping_patience'], restore_best_weights=True)

# Trainieren
history = model.fit(
    X, y,
    epochs=allg['train_epochs'],
    batch_size=allg['batch_size'],
    validation_split=allg['validation_split'],
    callbacks=[early_stop],
    verbose=1
)

C:\Users\ChangChi\AppData\Roaming\Python\Python313\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_3 (Conv1D)               │ (None, 126, 64)        │         1,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_4 (Conv1D)               │ (None, 124, 32)        │         6,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_4 (LSTM)                   │ (None, 50)             │        16,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,851 (93.17 KB)

 Trainable params: 23,851 (93.17 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/1000
 882/1073 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.5131 - loss: 0.6937

KeyboardInterrupt: 

In [10]:
# Mini
# Features bestimmen
n_features = X.shape[2]

# Modell erstellen
mini_model = create_small_cnn_lstm(
    seq_len,
    n_features, 
    dropout_rate=allg['dropout_rate'], 
    use_dropout=allg['use_dropout'])

mini_model.summary()

# EarlyStopping
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=allg['early_stopping_patience'],
    restore_best_weights=True
)

# Trainieren
history_mini = mini_model.fit(
    X, y,
    epochs=allg['train_epochs'],    # z.B. 1000
    batch_size=allg['batch_size'],  # z.B. 32
    validation_split=allg['validation_split'],  # z.B. 0.2
    callbacks=[early_stop],
    verbose=1
)


C:\Users\ChangChi\AppData\Roaming\Python\Python313\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 126, 32)        │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 126, 64)        │        24,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 126, 64)        │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 64)             │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 93,505 (365.25 KB)

 Trainable params: 93,505 (365.25 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/1000
537/537 ━━━━━━━━━━━━━━━━━━━━ 36s 64ms/step - accuracy: 0.5217 - loss: 0.6925 - val_accuracy: 0.5364 - val_loss: 0.6914
Epoch 2/1000
537/537 ━━━━━━━━━━━━━━━━━━━━ 33s 62ms/step - accuracy: 0.5235 - loss: 0.6920 - val_accuracy: 0.5374 - val_loss: 0.6910
Epoch 3/1000
537/537 ━━━━━━━━━━━━━━━━━━━━ 33s 62ms/step - accuracy: 0.5226 - loss: 0.6918 - val_accuracy: 0.5374 - val_loss: 0.6906
Epoch 4/1000
537/537 ━━━━━━━━━━━━━━━━━━━━ 33s 62ms/step - accuracy: 0.5254 - loss: 0.6916 - val_accuracy: 0.5365 - val_loss: 0.6911
Epoch 5/1000
537/537 ━━━━━━━━━━━━━━━━━━━━ 34s 64ms/step - accuracy: 0.5252 - loss: 0.6917 - val_accuracy: 0.5378 - val_loss: 0.6908
Epoch 6/1000
537/537 ━━━━━━━━━━━━━━━━━━━━ 35s 65ms/step - accuracy: 0.5254 - loss: 0.6916 - val_accuracy: 0.5359 - val_loss: 0.6908
Epoch 7/1000
537/537 ━━━━━━━━━━━━━━━━━━━━ 53s 99ms/step - accuracy: 0.5248 - loss: 0.6914 - val_accuracy: 0.5378 - val_loss: 0.6895
Epoch 8/1000
537/537 ━━━━━━━━━━━━━━━━━━━━ 63s 118ms/step - accuracy: 0.5249 

In [17]:
print(np.mean(X))
history_mini.save("models/Heusc_vMini_20250913_1m.keras")


-0.005478326381735886


AttributeError: 'History' object has no attribute 'save'